In [1]:
## Processing proteomic and genomic data for jurkat ##
# 0. Prepare Mapping Files 
# 1. Prepare Isoform Information Table from SQUANTI Output
# 2. Make Gene Length Statistics File
# 3. Abundace Data by Gene
# 4. Include PolyA Tail Info
# 5. Proteomics Analysis

#### Import Modules ####
# Python Modules #
import numpy as np
import pandas as pd 
from pathlib import Path

# Custom Modules #
from gen_maps import GenMap
from squantitable import sqtab
from make_gene_length_table import GenLenTab 
from MMprocess import MMproc

#### Input Files ####
gtf_file = './../jurkat_analysis/a_gencode_gene_models/gencode.v35.annotation.gtf'
squanti_out = './../jurkat_analysis/a_SQANTI3_out/jurkat_classification.txt'
fa_file = './../jurkat_analysis/a_gencode_gene_models/gencode.v35.pc_transcripts.fa'
tpm_file =  '../jurkat_analysis/a_jurkat_tpms_kallisto/c_gene_and_iso_kallisto_tables/a_jurkat_gene_kallisto.tsv'
yangpolyA = '../jurkat_analysis/a_polyA-_data/a_yang_et_al/a_genes_without_polyA_tails_from_Yang_et_al.tsv' # file prepared manually
ribodep_tpm = '../jurkat_analysis/a_polyA-_data/b_ribodeplete_jurkat_geo/d_kallist_table_rdeplete_jurkat.tsv' # expects normalized data
pbacc_to_gene_file = './uniprot_acc_to_gencode_gene.tsv'


## MM Inputs ##
mm_genecode_out = './../Map PacBio Gene Space/New_ForGloria/GENCODE_New_28Tryp/Task2-SearchTask/AllProteinGroups.tsv'
mm_uniprot_out = './../Map PacBio Gene Space/ForGloria/28FractionBUTrypsin/UniProtReviewed/Task2-SearchTask/AllProteinGroups.tsv'
mm_pacbio_out = './../Map PacBio Gene Space/ForGloria/28FractionBUTrypsin/PacBio_new/Task2-SearchTask/AllProteinGroups.tsv'

## Optional Inputs ##
# Can be either input or output locations
# If file exists, use as input. If file does not exist, use to custimize output location. 
ensg_to_gene = "./ensg_to_gene.tsv"
enst_to_trans = "./enst_to_trans.tsv"
ensp_to_gene = "./ensp_to_gene.tsv"
trans_to_gene = "./trans_to_gene.tsv"
gene_len_stats_tab =  './gen_len_stats.tsv'
gen_isolen_tab = "./gene_len.tsv" 

#### Output Files ####
squantitab = "./squanti_isoform_tab.tsv"
gen_level_tab = './gene_based_info.tsv'
comparison_tab = "./comparison_table.tsv"

#### TODO ###
# Separate Transcriptomic Data from Genomic Data
# Clean up inputs and outputs list 

####



In [2]:
#### Part 0: Preparing Maps, Files, etc. ####

## Find ENSG -> Gene, ENST -> Transcript and genename -> ENSP tables ##
maps = [ensg_to_gene, enst_to_trans, ensp_to_gene, trans_to_gene]
create_files = []
for i in maps:
    if Path(i).is_file()==False:
        create_files.append(i.replace('.tsv', '').replace('./', ''))

# If the files are not found, make them 
if not create_files:
    print("All mapping files have been prepared")
else:
    GenMap(gtf_file, create_files)


All mapping files have been prepared


In [3]:
#### Part 1 : Prepare Isoform Information Table from SQUANTI Output ####
sq_isotab = sqtab(squanti_out, ensg_to_gene, enst_to_trans, squantitab)
# TODO: Some pb_acc map to more than 1 gene

Isoform Table from Squanti output has been prepared


In [4]:
#### Part 2: Make Gene Length Statistics File ####
gene_len_stats = GenLenTab(fa_file, gen_len_stats_tab = './gen_len_stats.tsv',gen_isolen_tab='./gen_len.tsv')


The gene statistics table has already been prepared. Skipping this step


In [5]:
#### Part 3: Abundace Data by Gene ####
# TODO: Make into function?

## Pacbio Abundace from SQUANTI output ##
#sum(cpm per isoform) -> cpm per gene
cpm_data = sq_isotab[['gene', 'cpm']]
cpm_by_gene = cpm_data.groupby(['gene']).agg(cpm = ('cpm', 'sum')).reset_index(level=['gene'])

## Illumina Abundance from Kallisto output ##
tpm_by_gene = pd.read_csv(tpm_file, delimiter='\t')
tpm_by_gene['gene'] = tpm_by_gene['gene'].str.replace('-', '_')

## Merge Abundace Data ##
abund = pd.merge(cpm_by_gene, tpm_by_gene, how='outer', on='gene').fillna(0)

# Make sure there is no gene mapping issues. 
# TODO: Find lengths for the pb_only genes
test = pd.merge(cpm_by_gene, tpm_by_gene, how='inner', on='gene')
pb_only = cpm_by_gene[~cpm_by_gene['gene'].isin(test['gene'])]

## Merge with Gene Length Data and Save ##
gen_lenab = pd.merge(gene_len_stats, abund, how="outer", on='gene')

In [6]:
#### Part 4: Include PolyA Tail Info ####

## Yang et al ##
# TODO: Look at how to analyze PolyA better 
yang = pd.read_csv(yangpolyA, sep='\t')
ygen = gen_lenab.assign(is_polyA=~gen_lenab.gene.isin(yang.Gene))

## Ribodepletion Data ##
ribo = pd.read_csv(ribodep_tpm, sep='\t')
rgen = ribo.groupby(['gene']).agg(rtpm=('tpm', 'sum')).reset_index()
rgen['log(rtpm+1)'] = np.log10(rgen['rtpm'] + 1)

# Add 1 to tpm data #
tpm_by_gene['log(tpm+1)'] = np.log10(tpm_by_gene['tpm'] + 1)

# Merge and calc rtpm/tpm on log scale
pA = pd.merge(rgen, tpm_by_gene, how = 'outer', on='gene')
pA['ratio'] = pA['log(rtpm+1)']/pA['log(tpm+1)']
npA = pA[pA['ratio']>120].reset_index()

# Add PolyA info to Gene Length table 
pA_gen = ygen.assign(r_ispolyA =~ ygen.gene.isin(npA.gene))

# Output Gene-Level Table 
pA_gen.to_csv(gen_level_tab, sep='\t', index=False, na_rep=0)

In [15]:
#### Part 5: Proteomics Analysis ####

# Analyze Genecode Data and Merge #
gencode_out = MMproc(mm_genecode_out, 'genecode', trans_to_gene)
pGen = pd.merge(pA, gencode_out, how='outer', on='gene')

# Analyze Uniprot Data and Merge # 
uniprot_out = MMproc(mm_uniprot_out, 'uniprot', pbacc_to_gene_file)
pUn = pd.merge(pGen, uniprot_out, how='outer', on='gene')

# Analyze PacBio Data and Merge
pacbio_out = MMproc(mm_uniprot_out, 'pacbio')
pPac = pd.merge(pUn, pacbio_out, how='outer', on='gene')

# Read out Table #
pPac.to_csv(comparison_tab, sep='\t', index=False)


(7553, 6)
(7469, 6)
(7515, 6)
